## ENGRI 1120 Mole Balance Flux Balance Analysis for Chips in Series

<center>
    <img src="figs/Fig-Chips-in-Series.pdf" style="align:right; width:70%">
</center>

### Introduction
Suppose the liquid-phase enzyme-catalyzed production of product $P$ and bi-product $C$ is run on a well-mixed microfluidic chip with two input channels and a single output channel. The enzymes that carry out the chemistry are stable and tethered to the chip. 

Multiple chips are placed in series where the output stream of chip $j-1$ is fed into one of the input channels of chip $j$. 

Compute the optimal extents of reaction and the output channel composition using flux balance analysis for the chips in series configuration 

__Assumptions__:
* The chip is well-mixed and operates at steady-state
* The reaction volume of the chip is $V=100\mu{L}$.
* The chip is isothermal and isobaric and operates at the optimal temperature for the enzymes on the chip

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example/Manifest.toml`


In [2]:
# load reqd packages and set paths -
using PrettyTables
using GLPK

# setup paths -
const _ROOT = pwd();

#### Load the example code library
The call to the `include` function loads the `ENGRI-1120-Example-CodeLib.jl` library into the notebook; the library contains functions that we can use during the example. In particular, it contains the function:

* The `compute_optimal_extent(stoichiometric_matrix::Array{Float64,2}, default_bounds_array::Array{Float64,2},
    species_bounds_array::Array{Float64,2}, objective_coefficient_array::Array{Float64,1}; min_flag::Bool = true, θ::Float64 = 0.1) -> Tuple` function calls the [GLPK](https://www.gnu.org/software/glpk/) linear program solver. The `results` tuple contains several things, but the important ones are `calculated_flux_array`, `objective_value`, and the status/exit flags `status_flag` and `exit_flag` (which let us know if the solver successfully found a solution.

In [3]:
include("ENGRI-1120-Example-CodeLib.jl");

### a) Build the stoichiometric matrix $S$

In [4]:
# Setup a collection of reaction strings -
reaction_array = Array{String,1}()

# encode the reactions -
# internal reactions -
push!(reaction_array,"v₁,A₁+x,B+y,false")
push!(reaction_array,"v₂,B,P,false")
push!(reaction_array,"v₃,A₂+y,C+x,false")

# compute the stoichiometric matrix -
# the optional expand arguement = should we split reversible reactions? (default: false)
(S, species_name_array, reaction_name_array) = build_stoichiometric_matrix(reaction_array; 
    expand=false);

In [5]:
(ℳ, ℛ) = size(S);

### b) Solve the FBA problem for the first chip

In [6]:
# setup the bounds array -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 100.0; # set a default value for the *upper* bound on the flux

# setup the species bounds array -
# we know from out theory, that that the lower bound is -1*sum of the inputs 
ṅ₁ = zeros(ℳ);
ṅ₂ = zeros(ℳ);

# suppose we supply Ax in stream 1, and Bx in stream 2
ṅ₁[1] = 20.0; # supply A₁ -
ṅ₂[2] = 5.0;  # supply A₂ -

# setup -
species_bounds_array = [-1*(ṅ₁ .+ ṅ₂) 1000.0*ones(ℳ)];

# setup the objective vector -
c = zeros(ℛ);
c[2] = -1.0; # why is the negative?

# Call GLPK -
results = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, c);

# check: exit_flag = 0 and status_flag = 5 indicate a succesful soln -
println("Exit flag: $(results.exit_flag) and status flag: $(results.status_flag)")

# get the reaction extent vector -
ϵ̇ = results.calculated_flux_array;

# compute the output compostion for chip 1
ṅ₃_chip_1 = ṅ₂ + ṅ₁ + S*ϵ̇;

Exit flag: 0 and status flag: 5


### c) Setup and solve the FBA problem for chips $i=2,3,\dots,N$

In [7]:
# setup calculation for chips i = 2,....,N
N = 10 # number of chips

# initialize some space to store the mol flow rates -
series_mol_state_array = zeros(ℳ,N)
exit_flag_array = Array{Int64,1}()
status_flag_array = Array{Int64,1}()

# the initial col of this array is the output of from chip 1
for species_index = 1:ℳ
    series_mol_state_array[species_index,1] = ṅ₃_chip_1[species_index]
end

# assumption: we *always* feed A₂ into port 2 - so we only need to update the input flow into port 1
n_dot_input_stream_2 = ṅ₂;

for chip_index = 2:N

    # update the input into the chip -
    n_dot_input_port_1 = series_mol_state_array[:, chip_index - 1] 		# the input to chip j comes from j - 1

    # setup the species bounds array -
    species_bounds_next_chip = [-1.0*(n_dot_input_port_1.+ n_dot_input_stream_2) 1000.0*ones(ℳ,1)]

    # run the optimal calculation -
    result_next_chip = compute_optimal_extent(S, flux_bounds_array, species_bounds_next_chip, c);

    # grab the status and exit flags ... so we can check all is right with the world ...
    push!(exit_flag_array, result_next_chip.exit_flag)
    push!(status_flag_array, result_next_chip.status_flag)

    # Get the flux from the result object -
    ϵ_dot_next_chip = result_next_chip.calculated_flux_array

    # compute the output from chip j = chip_index 
    n_dot_out_next_chip = (n_dot_input_port_1 + n_dot_input_stream_2 + S*ϵ_dot_next_chip);

    # copy this state vector into the state array 
    for species_index = 1:ℳ
        series_mol_state_array[species_index, chip_index] = n_dot_out_next_chip[species_index]
    end

    # go around again ...
end

In [8]:
# build chip table -
chip_table_data = Array{Any,2}(undef, ℳ, N+2);

# in the first col, put the species labels -
for species_index ∈ 1:ℳ
    chip_table_data[species_index, 1] = species_name_array[species_index];
end

# input to chip 1 -
for species_index ∈ 1:ℳ
    chip_table_data[species_index, 2] = ṅ₁[species_index]+ṅ₂[species_index];
end

for chip_index ∈ 1:N
    for species_index ∈ 1:ℳ
        chip_table_data[species_index, chip_index+2] = series_mol_state_array[species_index, chip_index];
    end
end

# build header data -
chip_table_header = Array{String,1}();
push!(chip_table_header,"Species");
push!(chip_table_header,"input");
for chip_index ∈ 1:N
    push!(chip_table_header,"chip-$(chip_index)");
end

chip_table_header_units = Array{String,1}();
push!(chip_table_header_units,"");
push!(chip_table_header_units,"mol/time");
for chip_index ∈ 1:N
    push!(chip_table_header_units,"mol/time");
end

# show table -
pretty_table(chip_table_data; header = (chip_table_header, chip_table_header_units))

┌─────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ Species │    input │   chip-1 │   chip-2 │   chip-3 │   chip-4 │   chip-5 │   chip-6 │   chip-7 │   chip-8 │   chip-9 │  chip-10 │
│         │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │ mol/time │
├─────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│      A₁ │     20.0 │     15.0 │     10.0 │      5.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │
│      A₂ │      5.0 │      0.0 │      0.0 │      0.0 │      0.0 │      5.0 │     10.0 │     15.0 │     20.0 │     25.0 │     30.0 │
│       B │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │      0.0 │
│       C │      0.0 │      5.0 │     10.0 │     15.0 │     20.0 │   